Execute the original TankBind model, and try to find the differences in outputs between the old and the new model

In [1]:
%pwd

'/fs/gpfs41/lv11/fileset01/pool/pool-marsot'

In [2]:
%cd /fs/gpfs41/lv11/fileset01/pool/pool-marsot/tankbind_philip/TankBind/tankbind/

/fs/gpfs41/lv11/fileset01/pool/pool-marsot/tankbind_philip/TankBind/tankbind


In [3]:
import sys; sys.path.append("/fs/pool/pool-marsot")

In [4]:
import tankbind_philip.TankBind.tankbind as tankbind_og

In [5]:
from tankbind_philip.TankBind.tankbind.model import IaBNet_with_affinity

In [6]:
model_og = IaBNet_with_affinity()

In [23]:
model_og.conv_compound

GIN(
  (layers): ModuleList(
    (0): GraphIsomorphismConv(
      (mlp): MultiLayerPerceptron(
        (layers): ModuleList(
          (0): Linear(in_features=56, out_features=128, bias=True)
          (1): Linear(in_features=128, out_features=128, bias=True)
        )
      )
      (edge_linear): Linear(in_features=19, out_features=56, bias=True)
    )
    (1): GraphIsomorphismConv(
      (mlp): MultiLayerPerceptron(
        (layers): ModuleList(
          (0): Linear(in_features=128, out_features=56, bias=True)
          (1): Linear(in_features=56, out_features=56, bias=True)
        )
      )
      (edge_linear): Linear(in_features=19, out_features=128, bias=True)
    )
    (2): GraphIsomorphismConv(
      (mlp): MultiLayerPerceptron(
        (layers): ModuleList(
          (0): Linear(in_features=56, out_features=128, bias=True)
          (1): Linear(in_features=128, out_features=128, bias=True)
        )
      )
      (edge_linear): Linear(in_features=19, out_features=56, bias=Tru

In [8]:
from bindbind.models.model import TankBindModel

In [9]:
model = TankBindModel()

Protein embedding

In [14]:
#replace model weights with original model weights
for (name1, param1), (name2, param2) in zip(model.protein_embedding.named_parameters(), model_og.conv_protein.named_parameters()):
    param1.data = param2.data

Compound embedding

In [14]:
import torch

In [20]:
model.compound_embedding

GINE(
  (gnn_node): GNN_node(
    (atom_encoder): AtomEncoder(
      (linear): Linear(in_features=128, out_features=128, bias=True)
    )
    (convs): ModuleList(
      (0-4): 5 x GINConv()
    )
    (batch_norms): ModuleList(
      (0-4): 5 x BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
)

In [15]:
torch.__version__

'2.3.1'